# Engine


Here is an overview of the engine module, explaining where data is stored, the structure of database tables, and how rules and queries operate within the system.

---

#### Table of Contents

1. [Data Storage](#data-storage)
2. [Database Table Structure](#database-table-structure)
3. [Rules and Queries](#rules-and-queries)
---

## Data Storage

### Where is all the data about the relations stored?

Data about relations is stored in a SQLite3 database. 

> **SQLite3**: A module in Python's standard library that offers a lightweight and self-contained relational database management system. It enables creating, connecting to, and manipulating SQLite databases using Python.

---

## Database Table Structure

### What specific data is stored in the db tables, and what is the structure of these tables?

Whenever a relation is defined, a corresponding table is created in the database. <br>
The table has the same name as the relation, and its columns correspond to the variables in the relation—i.e., `len(term_list)`.
the names of columns in the database don't have an informative name and there is no need because from Python's perspective it doesn't understand what the relation or its values mean, it just sorts information in the way they were provided, for example brothers("Jack","Michael") from Python's perspective it just sees that "Jack" is the first term so it puts it in the col0 and "Michael" is the second so it gets added to col1.

#### Example

For example, if we define a new relation `brothers(str, str)`:

```prolog
%%rgxlog
new brothers(str, str)
```

The engine will create for us a new table for relation `brothers` with two columns called col0 and col1.

## Rules and Queries

### How Do Rules and Queries Work?

When a new rule is added to the system, an empty table is created in the database. <br> 
This table remains empty initially because multiple rules can share the same rule head. <br> 
During the querying process, the engine iterates through the list of rules, and the query result is a logical OR of these rules.

#### Example

For instance, if we define the rules:

```prolog
A(X,Y) <- B(X,Y)
A(X,Y) <- C(X,Y)
```

result would include all tuples that satisfy either `B(X, Y)` OR `C(X, Y)`.

---

## Queries

### Query Decomposition and Rule Breakdown

To understand querying in-depth, it's beneficial to break down a rule into its constituent parts. 

#### Example: Grandparent Rule

For example, let's consider the rule:

```prolog
grandparent(X, Z) <- parent(X, Y), parent(Y, Z)
```
*the rule head*: grandparent(X,Z) <br>
*the rule body*: relations: [parent(X,Y), parent(Y,Z)]

There are 4 operators in the engine that get called when breaking that rule, we will break down each one of them below.

:::{.callout-note}
Each operator yields a relation that is added to the database temporarily. (deleted when we're done querying).
:::

In [ ]:
#| default_exp engine

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#| hide
#| export
from nbdev.showdoc import show_doc
from fastcore.basics import patch

In [ ]:
#| export
import logging
import sqlite3 as sqlite
import tempfile
import pandas as pd
from abc import ABC, abstractmethod
from itertools import count
from jinja2 import Template
from pathlib import Path
from typing import Iterable, Optional, Set, Tuple, Any, List, Union, Dict, no_type_check, Sequence, Callable
from rgxlog.ast_node_types import RelationDeclaration, AddFact, RemoveFact, Query, IERelation, Relation
from rgxlog.primitive_types import Span, DataTypes, DataTypeMapping
from rgxlog.ie_function import IEFunction
from rgxlog.general_utils import strip_lines, string_to_span, get_free_var_to_relations_dict, get_output_free_var_names, extract_one_relation
from rgxlog.utils import patch_method

Installation NLP failed


In [ ]:
#| export
# rgx constants
RESERVED_RELATION_PREFIX = "__rgxlog__"

FALSE_VALUE: List = []
TRUE_VALUE: List[Tuple] = [tuple()]

logger = logging.getLogger(__name__)

In [ ]:
#| export
#| hide
import functools

def log_function_call(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling function '{func.__name__}' with arguments:")
        if args:
            print(f"Positional arguments: {args}")
        if kwargs:
            print(f"Keyword arguments: {kwargs}")
        result = func(*args, **kwargs)
        print(f"Function '{func.__name__}' finished executing")
        return result
    return wrapper

#| hide
#### RgxlogEngineBase

In [ ]:
#| export
class RgxlogEngineBase():
    """
    An abstraction for a rgxlog execution engine, used by `GenericExecution`.
    it includes relational algebra operators like join and project, database modification operators like
    `add_fact` (insert) and `remove_fact` (delete), and rgxlog-specific operators like `compute_ie_relation`.
    """

    def __init__(self) -> None:
        super().__init__()

    @abstractmethod
    def declare_relation_table(self, 
                    relation_decl: RelationDeclaration # a relation declaration
                    ) -> None:
        """
        Declares a relation in the rgxlog engine.
        """
        pass

    @abstractmethod
    def add_fact(self, 
                fact: AddFact # the fact to be added
                ) -> None:
        """
        Adds a fact to the rgxlog engine.
        """
        pass

    @abstractmethod
    def remove_fact(self, 
                    fact: RemoveFact # the fact to be removed
                    ) -> None:
        """
        Removes a fact from the rgxlog engine.
        """
        pass

    @abstractmethod
    def query(self, 
                query: Query # a query for the rgxlog engine
                ) -> List[Tuple]: # a list of tuples that are the query's results
        """
        Queries the rgxlog engine.
        Outputs a preformatted query result, e.g. [("a",5),("b",6)].
        notice that `query` isn't a string; it's a `Query` object which inherits from `Relation`. <br>
        for example, parsing the string `?excellent("bill","ted")` yields the following `Query`:

        ```prolog
        relation_name = excellent
        term_list = ["bill", "ted"]
        type_list = [DataTypes.string, DataTypes.string]
        ```
        """
        pass

    @abstractmethod
    def remove_tables(self, 
                tables_names: Iterable[str] # tables to remove
                ) -> None:
        """
        Removes all the tables inside the input from sql.
        """
        pass

    @abstractmethod
    def clear_relation(self, 
                    table: str # tables to reset
                    ) -> None:
        """
        Resets the table (deletes all its tuples).
        """
        pass

    def clear_tables(self, 
                tables_names: Iterable[str] # tables to reset
                ) -> None:
        """
        Resets all the tables inside the input (deletes all their tuples).
        """
        for table in tables_names:
            self.clear_relation(table)

    @abstractmethod
    def get_table_len(self, 
                table: str # name of a table
                    ) -> int: # number of tuples inside the table
        pass

    @abstractmethod
    def compute_ie_relation(self, 
                ie_relation: IERelation, # an ie relation that determines the input and output terms of the ie function
                ie_func: IEFunction, # the data for the ie function that will be used to compute the ie relation
                bounding_relation: Optional[Relation] # a relation that contains the inputs for ie_funcs. the actual input needs to be queried from it
                ) -> Relation: # a normal relation that contains all of the resulting tuples in the rgxlog engine
        """
        Computes an information extraction relation, returning the result as a normal relation.

        since ie relations may have input free variables, we need to use another relation to determine the inputs
        for ie_relation.
        Each free variable that appears as an ie_relation input term must appear at least once in the
        bounding_relation terms.
        Should the ie relation not have any input free variables, bounding_relation can (but not must) be None

        note the an ie relation is effectively two relations: a relation that defines the inputs to an ie function
        and a relation that filters the output of the ie functions.
        seeing that, we can define a general algorithm for this function:

        1. using the input relation to the ie function, filter "bounding_relation" into a relation who's
        tuples are all of the inputs to the ie function. if the input relation has no free variable terms,
        it is basically a fact, i.e., a single tuple, meaning we could skip this step and just use that tuple
        as an input instead.

        2. run the ie function on each one of the tuples of the relation we created in step 1, and save the
        results to a new relation (the output relation)

        3. use the output relation of the ie function, filter the results we got in step 2 to a new relation

        example:
        let's follow the steps for the ie relation "RGX<X,Y>->(Z,[1,2))" with the bounding relation "bind(X,Y,W)":

        1. input_relation(X,Y) <- bind(X,Y,W)  # filter the bounding relation tuples into the input relation

        2. for each tuple in input_relation(X,Y), use it as an input for RGX(x,y) and save the result to a new
        relation called output_relation

        3. filter the outputs of the ie function into a relation like this
        final_ie_result(X,Y,Z) <- input_relation(X,Y), output_relation(Z,[1,2))
        for details on this join operation see "RgxlogEngineBase.join_relations"

        note that "final_ie_result" is defined using only free variables, as an ie relation in rgxlog is merely
        a part of a rule body, which serves to define the rule head relation. the rule head only "cares" about
        free variables, so we can throw away all of the columns defined by constant terms.
        """
        pass

    @abstractmethod
    def _convert_relation_term_to_string_or_int(self, 
                                                datatype: DataTypes, # the type of the term
                                                  term: Any # the term object itself
                                                  ) -> Union[str, int]: # string/int representation
        """
        Return the string/int representation of a relation term, e.g. "[1,4)"
        """
        pass

    @abstractmethod
    def operator_select(self, 
                        relation: Relation, # the relation from which we select tuples
                        select_info: Set[Tuple[int, Any, DataTypes]], # set of tuples. each tuple contains the index of the column, the value to select and the type of the column
                        *args: Any
                        ) -> Relation: # a filtered relation
        """
        The `operator_select` function operates on a relation, which is represented as a database table, and a set of constant variables. Its primary objective is to filter tuples from the given relation based on certain conditions and return the filtered relation.
        For instance, consider the rule: 
        ```prolog
        brothers_of_jack(X) <- brothers(X, "Jack")
        ```
        The goal is to print all the brothers of "Jack" by retrieving tuples from the "brothers" relation table in the database where "Jack" appears on the right side of the tuple.
        The operator_select function serves precisely this purpose.
        """
        pass

    @abstractmethod
    def operator_join(self, 
                      relations: List[Relation], # a list of normal relations
                      *args: Any
                      ) -> Relation: # a new relation as described above
        """
        The `operator_join` function operates on a list of relations and its primary objective is to generate a logical AND operator among the tables of those relations within the database. <br>
        This function plays a crucial role in situations where the following rule is defined: <br> 

        ```prolog
        grandparent(X,Z) <- parent(X,Y), parent(Y,Z)
        ```

        The purpose of creating a relation through the `operator_join` function is to establish a connection between the columns containing all possible combinations of triplets (X, Y, Z) where both `parent(X,Y)` and `parent(Y,Z)` evaluate to true. <br> 
        The reason for having three columns instead of two is to ensure that we have an answer for every query pertaining to the grandparent rule. <br>
        With three free variables, it becomes essential to capture all possible combinations in order to provide comprehensive responses to queries. <br> 
        Subsequently, the columns required by a specific query can be projected at a later stage. <br>
        """
        pass

    @abstractmethod
    def operator_project(self, 
                         relation: Relation, # the relation on which we project
                         project_vars: List[str], # a list of variables on which we project
                           *args: Any
                           ) -> Relation: # the projected relation
        """
        The `operator_project` function operates on a relation, which is represented as a database table, and a list of project variables. <br> 
        Its primary objective is to yield a relation with the projected variables of the src_relation.

        Continuing from the previous example, once the select relation table is added to the database, it consists of two columns where "Jack" appears on the right-hand side. <br>
        However, the purpose of the query is to print only a single column containing the brothers of "Jack", without including the right column that holds the value "Jack". <br>

        To address this requirement, the `operator_project` comes into play. <br> 
        Its primary objective is to operate on the relation resulting from the select operation and generate another relation where "Jack" no longer appears on the right-hand side.
        """
        pass

    @abstractmethod
    def operator_union(self, 
                       relations: List[Relation], # a list of relations to unite
                       *args: Any
                       ) -> Relation: # the united relation
        """
        a rule can be defined multiple times as long as it shares the same rule head. To illustrate this, let's consider the following example:

        Suppose we have the rules:
        ```prolog
        A(X,Y) <- B(X,Y)
        A(X,Y) <- C(X,Y)
        ```
        If we execute the query `?A(X,Y)`, the engine will produce results that include tuples from both `B(X,Y)` and `C(X,Y)` relations. 

        Here's how the engine handles this scenario: <br>
        1. **Rule Matching**: The engine identifies all the rules that have the same rule head (`A(X,Y)`) as the queried relation. <br>
        2. **Rule Execution**: For each matched rule, the engine executes all the steps described earlier, such as assigning temporary names, performing joins, and creating intermediate relations, selecting and filtering. <br>
        3. **Intermediate Relations**: At this stage, we have intermediate relations generated for each rule, containing the relevant tuples based on the rule's body. <br>
        4. **Operator Union**: To consolidate the results from all the rules into a single unified relation, the operator union is applied. This operation combines the tuples from all the intermediate relations. <br>
       
        The `operator_union` ensures that the final unified relation contains all the tuples from the individual intermediate relations, effectively merging the results obtained from different rule instances with the same rule head.
        In summary, the engine processes each rule with the same rule head as the queried relation, performs the necessary steps for each rule individually, and then applies the operator union to merge the intermediate relations into a single unified relation, ensuring all relevant tuples are included in the final result.        @note: we assume that all the relations have same free_vars in the same order.

        """
        pass

    @abstractmethod
    def operator_copy(self, 
                src_rel: Relation, # the relation to copy from
                output_relation: Optional[Relation] = None, # if this is None, create a unique name for the output relation. otherwise, this will be the name of the output relation
                *args: Any) -> Relation: # the copied relation
        """
        Copies computed_relation to rule_relation.
        """
        pass

In [ ]:
show_doc(RgxlogEngineBase.declare_relation_table)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L64){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.declare_relation_table

>      RgxlogEngineBase.declare_relation_table
>                                               (relation_decl:rgxlog.ast_node_t
>                                               ypes.RelationDeclaration)

Declares a relation in the rgxlog engine.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation_decl | RelationDeclaration | a relation declaration |
| **Returns** | **None** |  |

::: {.callout-note}
if the relation is already declared does nothing.
:::

In [ ]:
show_doc(RgxlogEngineBase.add_fact)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L73){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.add_fact

>      RgxlogEngineBase.add_fact (fact:rgxlog.ast_node_types.AddFact)

Adds a fact to the rgxlog engine.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| fact | AddFact | the fact to be added |
| **Returns** | **None** |  |

In [ ]:
show_doc(RgxlogEngineBase.remove_fact)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L82){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.remove_fact

>      RgxlogEngineBase.remove_fact (fact:rgxlog.ast_node_types.RemoveFact)

Removes a fact from the rgxlog engine.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| fact | RemoveFact | the fact to be removed |
| **Returns** | **None** |  |

In [ ]:
show_doc(RgxlogEngineBase.query)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L91){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.query

>      RgxlogEngineBase.query (query:rgxlog.ast_node_types.Query)

Queries the rgxlog engine.
Outputs a preformatted query result, e.g. [("a",5),("b",6)].
notice that `query` isn't a string; it's a `Query` object which inherits from `Relation`. <br>
for example, parsing the string `?excellent("bill","ted")` yields the following `Query`:

```prolog
relation_name = excellent
term_list = ["bill", "ted"]
type_list = [DataTypes.string, DataTypes.string]
```

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| query | Query | a query for the rgxlog engine |
| **Returns** | **List[Tuple]** | **a list of tuples that are the query's results** |

In [ ]:
show_doc(RgxlogEngineBase.remove_tables)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L109){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.remove_tables

>      RgxlogEngineBase.remove_tables (tables_names:Iterable[str])

Removes all the tables inside the input from sql.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| tables_names | Iterable[str] | tables to remove |
| **Returns** | **None** |  |

In [ ]:
show_doc(RgxlogEngineBase.clear_relation)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L118){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.clear_relation

>      RgxlogEngineBase.clear_relation (table:str)

Resets the table (deletes all its tuples).

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| table | str | tables to reset |
| **Returns** | **None** |  |

In [ ]:
show_doc(RgxlogEngineBase.clear_tables)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L126){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.clear_tables

>      RgxlogEngineBase.clear_tables (tables_names:Iterable[str])

Resets all the tables inside the input (deletes all their tuples).

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| tables_names | Iterable[str] | tables to reset |
| **Returns** | **None** |  |

In [ ]:
show_doc(RgxlogEngineBase.get_table_len)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L136){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.get_table_len

>      RgxlogEngineBase.get_table_len (table:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| table | str | name of a table |
| **Returns** | **int** | **number of tuples inside the table** |

In [ ]:
show_doc(RgxlogEngineBase.compute_ie_relation)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L142){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.compute_ie_relation

>      RgxlogEngineBase.compute_ie_relation
>                                            (ie_relation:rgxlog.ast_node_types.
>                                            IERelation, ie_func:rgxlog.ie_funct
>                                            ion.IEFunction, bounding_relation:U
>                                            nion[rgxlog.ast_node_types.Relation
>                                            ,NoneType])

Computes an information extraction relation, returning the result as a normal relation.

since ie relations may have input free variables, we need to use another relation to determine the inputs
for ie_relation.
Each free variable that appears as an ie_relation input term must appear at least once in the
bounding_relation terms.
Should the ie relation not have any input free variables, bounding_relation can (but not must) be None

note the an ie relation is effectively two relations: a relation that defines the inputs to an ie function
and a relation that filters the output of the ie functions.
seeing that, we can define a general algorithm for this function:

1. using the input relation to the ie function, filter "bounding_relation" into a relation who's
tuples are all of the inputs to the ie function. if the input relation has no free variable terms,
it is basically a fact, i.e., a single tuple, meaning we could skip this step and just use that tuple
as an input instead.

2. run the ie function on each one of the tuples of the relation we created in step 1, and save the
results to a new relation (the output relation)

3. use the output relation of the ie function, filter the results we got in step 2 to a new relation

example:
let's follow the steps for the ie relation "RGX<X,Y>->(Z,[1,2))" with the bounding relation "bind(X,Y,W)":

1. input_relation(X,Y) <- bind(X,Y,W)  # filter the bounding relation tuples into the input relation

2. for each tuple in input_relation(X,Y), use it as an input for RGX(x,y) and save the result to a new
relation called output_relation

3. filter the outputs of the ie function into a relation like this
final_ie_result(X,Y,Z) <- input_relation(X,Y), output_relation(Z,[1,2))
for details on this join operation see "RgxlogEngineBase.join_relations"

note that "final_ie_result" is defined using only free variables, as an ie relation in rgxlog is merely
a part of a rule body, which serves to define the rule head relation. the rule head only "cares" about
free variables, so we can throw away all of the columns defined by constant terms.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| ie_relation | IERelation | an ie relation that determines the input and output terms of the ie function |
| ie_func | IEFunction | the data for the ie function that will be used to compute the ie relation |
| bounding_relation | Optional[Relation] | a relation that contains the inputs for ie_funcs. the actual input needs to be queried from it |
| **Returns** | **Relation** | **a normal relation that contains all of the resulting tuples in the rgxlog engine** |

In [ ]:
show_doc(RgxlogEngineBase._convert_relation_term_to_string_or_int)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L189){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase._convert_relation_term_to_string_or_int

>      RgxlogEngineBase._convert_relation_term_to_string_or_int
>                                                                (datatype:rgxlo
>                                                                g.primitive_typ
>                                                                es.DataTypes,
>                                                                term:Any)

Return the string/int representation of a relation term, e.g. "[1,4)"

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| datatype | DataTypes | the type of the term |
| term | Any | the term object itself |
| **Returns** | **Union[str, int]** | **string/int representation** |

In [ ]:
show_doc(RgxlogEngineBase.operator_select)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L199){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.operator_select

>      RgxlogEngineBase.operator_select
>                                        (relation:rgxlog.ast_node_types.Relatio
>                                        n, select_info:Set[Tuple[int,Any,rgxlog
>                                        .primitive_types.DataTypes]],
>                                        *args:Any)

The `operator_select` function operates on a relation, which is represented as a database table, and a set of constant variables. Its primary objective is to filter tuples from the given relation based on certain conditions and return the filtered relation.
For instance, consider the rule: 
```prolog
brothers_of_jack(X) <- brothers(X, "Jack")
```
The goal is to print all the brothers of "Jack" by retrieving tuples from the "brothers" relation table in the database where "Jack" appears on the right side of the tuple.
The operator_select function serves precisely this purpose.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation | Relation | the relation from which we select tuples |
| select_info | Set[Tuple[int, Any, DataTypes]] | set of tuples. each tuple contains the index of the column, the value to select and the type of the column |
| args | Any |  |
| **Returns** | **Relation** | **a filtered relation** |

In [ ]:
show_doc(RgxlogEngineBase.operator_join)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L216){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.operator_join

>      RgxlogEngineBase.operator_join
>                                      (relations:List[rgxlog.ast_node_types.Rel
>                                      ation], *args:Any)

The `operator_join` function operates on a list of relations and its primary objective is to generate a logical AND operator among the tables of those relations within the database. <br>
This function plays a crucial role in situations where the following rule is defined: <br> 

```prolog
grandparent(X,Z) <- parent(X,Y), parent(Y,Z)
```

The purpose of creating a relation through the `operator_join` function is to establish a connection between the columns containing all possible combinations of triplets (X, Y, Z) where both `parent(X,Y)` and `parent(Y,Z)` evaluate to true. <br> 
The reason for having three columns instead of two is to ensure that we have an answer for every query pertaining to the grandparent rule. <br>
With three free variables, it becomes essential to capture all possible combinations in order to provide comprehensive responses to queries. <br> 
Subsequently, the columns required by a specific query can be projected at a later stage. <br>

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relations | List[Relation] | a list of normal relations |
| args | Any |  |
| **Returns** | **Relation** | **a new relation as described above** |

:::{.callout-note}
It should be noted that the relation generated at this point represents the logical AND operation applied to all the relations within the rule's body relation, considering the free variables involved.
:::

In [ ]:
show_doc(RgxlogEngineBase.operator_project)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L236){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.operator_project

>      RgxlogEngineBase.operator_project
>                                         (relation:rgxlog.ast_node_types.Relati
>                                         on, project_vars:List[str], *args:Any)

The `operator_project` function operates on a relation, which is represented as a database table, and a list of project variables. <br> 
Its primary objective is to yield a relation with the projected variables of the src_relation.

Continuing from the previous example, once the select relation table is added to the database, it consists of two columns where "Jack" appears on the right-hand side. <br>
However, the purpose of the query is to print only a single column containing the brothers of "Jack", without including the right column that holds the value "Jack". <br>

To address this requirement, the `operator_project` comes into play. <br> 
Its primary objective is to operate on the relation resulting from the select operation and generate another relation where "Jack" no longer appears on the right-hand side.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation | Relation | the relation on which we project |
| project_vars | List[str] | a list of variables on which we project |
| args | Any |  |
| **Returns** | **Relation** | **the projected relation** |

In [ ]:
show_doc(RgxlogEngineBase.operator_union)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L254){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.operator_union

>      RgxlogEngineBase.operator_union
>                                       (relations:List[rgxlog.ast_node_types.Re
>                                       lation], *args:Any)

a rule can be defined multiple times as long as it shares the same rule head. To illustrate this, let's consider the following example:

Suppose we have the rules:
```prolog
A(X,Y) <- B(X,Y)
A(X,Y) <- C(X,Y)
```
If we execute the query `?A(X,Y)`, the engine will produce results that include tuples from both `B(X,Y)` and `C(X,Y)` relations. 

Here's how the engine handles this scenario: <br>
1. **Rule Matching**: The engine identifies all the rules that have the same rule head (`A(X,Y)`) as the queried relation. <br>
2. **Rule Execution**: For each matched rule, the engine executes all the steps described earlier, such as assigning temporary names, performing joins, and creating intermediate relations, selecting and filtering. <br>
3. **Intermediate Relations**: At this stage, we have intermediate relations generated for each rule, containing the relevant tuples based on the rule's body. <br>
4. **Operator Union**: To consolidate the results from all the rules into a single unified relation, the operator union is applied. This operation combines the tuples from all the intermediate relations. <br>

The `operator_union` ensures that the final unified relation contains all the tuples from the individual intermediate relations, effectively merging the results obtained from different rule instances with the same rule head.
In summary, the engine processes each rule with the same rule head as the queried relation, performs the necessary steps for each rule individually, and then applies the operator union to merge the intermediate relations into a single unified relation, ensuring all relevant tuples are included in the final result.        @note: we assume that all the relations have same free_vars in the same order.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relations | List[Relation] | a list of relations to unite |
| args | Any |  |
| **Returns** | **Relation** | **the united relation** |

In [ ]:
show_doc(RgxlogEngineBase.operator_copy)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L281){target="_blank" style="float:right; font-size:smaller"}

### RgxlogEngineBase.operator_copy

>      RgxlogEngineBase.operator_copy (src_rel:rgxlog.ast_node_types.Relation,
>                                      output_relation:Union[rgxlog.ast_node_typ
>                                      es.Relation,NoneType]=None, *args:Any)

Copies computed_relation to rule_relation.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| src_rel | Relation |  | the relation to copy from |
| output_relation | Optional[Relation] | None | if this is None, create a unique name for the output relation. otherwise, this will be the name of the output relation |
| args | Any |  |  |
| **Returns** | **Relation** |  | **the copied relation** |

#| hide
#### SqliteEngine

In [ ]:
#| export
class SqliteEngine(RgxlogEngineBase):
    """
    in this implementation of the engine, we use python's sqlite3, which allows creating an SQL database easily, without using servers.
    the engine is called from `GenericExecution`, and uses `run_sql` as an interface to the database, which queries/modifies a table.
    each `operator` method implements a relational algebra operator by constructing an SQL command and executing it.
    """

    # useful prefixes
    PROJECT_PREFIX = "project"
    JOIN_PREFIX = "join"
    COPY_PREFIX = "copy"
    SELECT_PREFIX = "select"
    UNION_PREFIX = "union"
    RELATION_COLUMN_PREFIX = "col"

    # sql constants
    SQL_SELECT = "SELECT DISTINCT"
    SQL_TABLE_OF_TABLES = "sqlite_master"
    SQL_SEPARATOR = "_"
    DATATYPE_TO_SQL_TYPE = {DataTypes.string: "TEXT", DataTypes.integer: "INTEGER", DataTypes.span: "TEXT"}
    DATABASE_SUFFIX = "_sqlite"
        
    # ~~ dunder methods ~~
    def __init__(self, 
                database_name: Optional[str] = None # open an existing database instead of a new one
                ):
        """
        Creates/opens an SQL database file + connection.
        """
        super().__init__()
        self.unique_relation_id_counter = count()

        self.df_filename = SqliteEngine._get_db_filename(database_name)
        logger.info(f"using database file: {self.df_filename}")

        self.sql_conn = sqlite.connect(self.df_filename)
        self.sql_cursor = self.sql_conn.cursor()

    def __del__(self) -> None:
        self.sql_conn.close()
 
    @staticmethod
    def _datatype_to_sql_type(datatype: DataTypes) -> str:
        return SqliteEngine.DATATYPE_TO_SQL_TYPE[datatype]

    @staticmethod
    def _get_free_variable_indexes(type_list: Sequence[DataTypes]) -> List[int]:
        return [i for i, term_type in enumerate(type_list) if (term_type is DataTypes.free_var_name)]

    @staticmethod
    def _assert_ie_output_properly_typed(
                    ie_input: Iterable, # the input of the ie function (used in the exception when the type check fails)
                    ie_output: Iterable, # an output of the ie function
                    ie_output_schema: Iterable, # the expected schema for ie_output
                    ie_relation: IERelation # the ie relation for which the output was computed (will be used to print an exception in case the output is not properly typed)
                    ) -> None:
        """
        Even though rgxlog performs typechecking during the semantic checks phase, information extraction functions
        are written by the users and could yield results that are not properly typed.
        this method asserts an information extraction function's output is properly typed.

        @raise TypeError: if there is output term of an unsupported type or the output relation is not properly typed.
        """

        # get a list of the ie output's term types
        ie_output_term_types = []
        for output_term in ie_output:
            if isinstance(output_term, int):
                output_type = DataTypes.integer
            elif isinstance(output_term, str):
                output_type = DataTypes.string
            elif isinstance(output_term, Span):
                # allow the user to return a span as either a tuple of length 2 or a datatypes.Span instance
                output_type = DataTypes.span
            else:
                # encountered an output term of an unsupported type
                raise TypeError(f'executing ie relation {ie_relation}\n'
                                f'with the input {ie_input}\n'
                                f'failed because one of the outputs had an unsupported term type\n'
                                f'the output: {ie_output}\n'
                                f'the invalid term: {output_term}\n'
                                f'the invalid term type: {type(output_term)}\n'
                                f'note that only strings, spans and integers are supported\n'
                                f'spans can be represented as a tuple of length 2 or as a datatypes.span instance')
            ie_output_term_types.append(output_type)

        # assert that the ie output is properly typed
        ie_output_is_properly_typed = ie_output_term_types == list(ie_output_schema)
        if not ie_output_is_properly_typed and len(ie_output_term_types) - len(ie_relation.input_term_list) != 0:
            raise TypeError(f'executing ie relation {ie_relation}\n'
                            f'with the input {ie_input}\n'
                            f'failed because one of the outputs had unexpected term types\n'
                            f'the output: {ie_output}\n'
                            f'the output term types: {ie_output_term_types}\n'
                            f'the expected types: {ie_output_schema}')

    @staticmethod
    def _convert_strings_to_spans_in_query_result(query_result: List[Tuple]) -> List[Tuple]:
        """
        convert strings that look like spans into spans
        @param query_result: the list of tuples which may contain strings that should be converted to spans
        @return: the same list, but with span-looking strings converted to `Span` objects
        """
        spanned_query_result = []
        for row in query_result:
            converted_row: List[Union[str, int, Span]] = []
            for value in row:
                if isinstance(value, str):
                    transformed_string = string_to_span(value)
                    if transformed_string is None:
                        converted_row.append(value)
                    else:
                        converted_row.append(transformed_string)
                else:
                    converted_row.append(value)

            spanned_query_result.append(tuple(converted_row))
        return spanned_query_result

    @staticmethod
    def _get_db_filename(database_name: Optional[Any]) -> str:
        if database_name:
            if not Path(database_name).is_file():
                raise IOError(f"database file: {database_name} was not found")
            return database_name

        temp_db_file = tempfile.NamedTemporaryFile(delete=False, suffix=SqliteEngine.DATABASE_SUFFIX)
        temp_db_file.close()
        return temp_db_file.name


 

In [ ]:
#| export
#| hide
# Helper method for testing
@patch_method
def table_to_dataframe(self : SqliteEngine ,name) -> pd.DataFrame:
    self.sql_cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    table_names = self.sql_cursor.fetchall()
    # Iterate over the table names and print the data from each table
    for table_name in table_names:
        if table_name[0] == name:
            query = f"SELECT * FROM {table_name[0]}"
            self.sql_cursor.execute(query)
            
            # Fetch all rows from the query result
            rows = self.sql_cursor.fetchall()

            # Get column names from the cursor description
            column_names = [description[0] for description in self.sql_cursor.description]

            # Create a Pandas DataFrame from the fetched rows and column names
            df = pd.DataFrame(rows, columns=column_names)
            
            return df

In [ ]:
#| export
#| hide
@patch_method
def print_sql(self: SqliteEngine):
    self.sql_cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    table_names = self.sql_cursor.fetchall()

    # Iterate over the table names and print the data from each table
    for table_name in table_names:
        # Execute a SELECT query to retrieve all rows from the current table
        self.sql_cursor.execute(f"SELECT * FROM {table_name[0]}")

        # Fetch all rows from the query result
        rows = self.sql_cursor.fetchall()

        # Print the table name
        print(f"Table: {table_name[0]}")

        # Print the retrieved data
        for row in rows:
            print(row)
        print()

In [ ]:
#| export
#| hide
@patch_method
def _run_sql_from_jinja_template(self: SqliteEngine, sql_template: str, template_dict: Optional[dict] = None) -> None:
    if not template_dict:
        template_dict = {}

    sql_command = Template(strip_lines(sql_template)).render(**template_dict)
    self._run_sql(sql_command)

In [ ]:
#| export
#| hide
@patch_method
def _run_sql(self: SqliteEngine, command: str, command_args: Optional[List] = None, do_commit: bool = False) -> List:
    logger.debug(f"sql {command=}")
    if command_args:
        logger.debug(f"...with args: {command_args}")

    if command_args:
        self.sql_cursor.execute(command, command_args)
    else:
        self.sql_cursor.execute(command)

    if do_commit:
        # save to file
        self.sql_conn.commit()

    return self.sql_cursor.fetchall()

In [ ]:
#| export
#| hide
@patch_method
def _get_col_name(self: SqliteEngine, col_id: int) -> str:
    return f'{SqliteEngine.RELATION_COLUMN_PREFIX}{col_id}'

In [ ]:
#| export
#| hide
@patch_method
def get_table_len(self: SqliteEngine, table_name: str) -> int:
    sql_command = f"SELECT COUNT(*) FROM {table_name}"
    table_len, = self._run_sql(sql_command)[0]
    return table_len

In [ ]:
#| export
#| hide
@patch_method
def declare_relation_table(self: SqliteEngine, 
                relation_decl: RelationDeclaration # the declaration info
                ):
    """
    Declares a relation as an SQL table, whose types are named t0, t1, ...
    if the relation is already declared, do nothing.
    """
    # create the relation table. we don't use an id because it would allow inserting the same values twice
    # note: to ignore duplicates, we can either use UNIQUE when creating the table, or DISTINCT when selecting.
    #  right now we use DISTINCT
    if self.is_table_exists(relation_decl.relation_name):
        return

    # note: sqlite can guess datatypes. if this causes bugs, use `{SqliteEngine._datatype_to_sql_type(relation_type)}`.
    col_names = [f"{self._get_col_name(i)}" for i in range(len(relation_decl.type_list))]
    template_dict = {"rel_name": relation_decl.relation_name, "col_names": col_names}
    sql_template = 'CREATE TABLE {{rel_name}} ({{col_names | join(", ")}})'

    self._run_sql_from_jinja_template(sql_template, template_dict)

In [ ]:
#| export
#| hide
@patch_method
def _create_unique_relation(self: SqliteEngine, 
                            arity: int, # the relation's arity
                            # will be used as a part of the relation's name
                            prefix: str = "" # for example: prefix='join' -> full name = `__rgxlog__join{counter}``
                            ) -> str: # the new relation's name
    """
    Declares a new relation with the requested arity in SQL, the relation will have a unique name.
    """
    # create the name of the new relation
    unique_relation_id = next(self.unique_relation_id_counter)
    if RESERVED_RELATION_PREFIX in prefix:
        # we don't want relations to be called __rgxlog__rgxlog__rgxlog...
        unique_relation_name = f'{prefix}{unique_relation_id}'
    else:
        unique_relation_name = f'{RESERVED_RELATION_PREFIX}{prefix}{unique_relation_id}'

    # in SQLite there's no typechecking so we just need to make sure that the schema has the correct arity
    unique_relation_schema = [DataTypes.free_var_name] * arity

    # create the declaration
    unique_relation_decl = RelationDeclaration(unique_relation_name, unique_relation_schema)

    # create the relation's SQL table, and return its name
    self.declare_relation_table(unique_relation_decl)
    return unique_relation_name

In [ ]:
#| export
#| hide
@patch_method
def _convert_relation_term_to_string_or_int(self: SqliteEngine, datatype: DataTypes, term: DataTypeMapping.term) -> Union[str, int]:
    if datatype is DataTypes.integer:
        assert isinstance(term, int), "an integer must be of int type"
        return term
    else:
        unquoted_term = str(term).strip('"')
        return f'"{unquoted_term}"'

In [ ]:
#| export
#| hide
@patch_method
def clear_relation(self: SqliteEngine, table_name: str) -> None:
    sql_command = f"DELETE FROM {table_name}"
    self._run_sql(sql_command)

In [ ]:
#| export
#| hide
@patch_method
def is_table_exists(self: SqliteEngine, 
                    table_name: str # the table which is checked for existence.
                    )-> bool: # True if it exists, else False.
    """
    Checks whether a table exists in the database. 
    """
    sql_check_if_exists = f"{SqliteEngine.SQL_SELECT} name FROM {SqliteEngine.SQL_TABLE_OF_TABLES} WHERE " f"type='table' AND name='{table_name}'"
    return bool(self._run_sql(sql_check_if_exists))

In [ ]:
show_doc(SqliteEngine.is_table_exists)

---

### SqliteEngine.is_table_exists

>      SqliteEngine.is_table_exists (table_name:str)

Checks whether a table exists in the database.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| table_name | str | the table which is checked for existence. |
| **Returns** | **bool** | **True if it exists, else False.** |

In [ ]:
#| export
#| hide
@patch_method
def remove_table(self: SqliteEngine, 
                table_name: str # the table to remove
                ):
    """
    Removes a table from the sql database, if it exists.
    """
    if self.is_table_exists(table_name):
        sql_command = f"DROP TABLE {table_name}"
        self._run_sql(sql_command)

In [ ]:
show_doc(SqliteEngine.remove_table)

---

### SqliteEngine.remove_table

>      SqliteEngine.remove_table (table_name:str)

Removes a table from the sql database, if it exists.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| table_name | str | the table to remove |

In [ ]:
#| export
#| hide
@patch_method
def remove_tables(self: SqliteEngine, 
            table_names: Iterable[str] # tables to remove
            ):
    """
    Removes the given tables from sql.
    """
    for table_name in table_names:
        self.remove_table(table_name)

In [ ]:
show_doc(SqliteEngine.remove_tables)

---

### SqliteEngine.remove_tables

>      SqliteEngine.remove_tables (table_names:Iterable[str])

Removes the given tables from sql.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| table_names | Iterable[str] | tables to remove |

In [ ]:
#| export
#| hide
@patch_method
def add_fact(self: SqliteEngine, 
            fact: AddFact # the fact to be added
            ) -> None:
    """
    Add a row into an existing sql table, based on `fact`'s terms and types
    """
    num_types = len(fact.type_list)
    col_names = [f"{self._get_col_name(i)}" for i in range(num_types)]
    col_values = [self._convert_relation_term_to_string_or_int(datatype, term) for datatype, term in
                    zip(fact.type_list, fact.term_list)]

    template_dict = {"col_values": col_values, "fact": fact, "col_names": col_names}

    sql_template = ("""
    INSERT INTO {{fact.relation_name}} ({{col_names | join(", ")}})
    VALUES ({{col_values | join(", ")}})
    """)

    self._run_sql_from_jinja_template(sql_template, template_dict)

#| hide
#### TEST add_fact

In [ ]:
#| hide
my_engine = SqliteEngine()

relation_yoyo = RelationDeclaration("yoyo", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation_yoyo)

facts_yoyo = [
    AddFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]),
    AddFact("yoyo", [10, "hello"], [DataTypes.integer, DataTypes.string]),
    AddFact("yoyo", [15, "world"], [DataTypes.integer, DataTypes.string])
]

for fact in facts_yoyo:
    my_engine.add_fact(fact)

# Expected output for yoyo
expected_output_yoyo_df = pd.DataFrame([(8,'hihi'),(10, 'hello'), (15, 'world')], columns=["col0", "col1"])

curr_yoyo_table = my_engine.table_to_dataframe("yoyo")
pd.testing.assert_frame_equal(curr_yoyo_table, expected_output_yoyo_df)

# Similar declarations for numbers and words
relation_numbers = RelationDeclaration("numbers", [DataTypes.integer])
relation_words = RelationDeclaration("words", [DataTypes.string])
my_engine.declare_relation_table(relation_numbers)
my_engine.declare_relation_table(relation_words)

# Facts for numbers and words
facts_numbers_words = [
    AddFact("numbers", [5], [DataTypes.integer]),
    AddFact("numbers", [10], [DataTypes.integer]),
    AddFact("words", ["apple"], [DataTypes.string]),
    AddFact("words", ["banana"], [DataTypes.string])
]

for fact in facts_numbers_words:
    my_engine.add_fact(fact)

# Expected output for numbers and words
expected_numbers_output_df = pd.DataFrame([5, 10],columns=["col0"])
expected_words_output_df = pd.DataFrame(['apple', 'banana'],columns=["col0"])

# Comparisons for numbers and words
curr_numbers_table = my_engine.table_to_dataframe("numbers")
pd.testing.assert_frame_equal(curr_numbers_table, expected_numbers_output_df)

curr_words_table = my_engine.table_to_dataframe("words")
pd.testing.assert_frame_equal(curr_words_table, expected_words_output_df)

# Similar declarations and comparisons for students and courses
relation_students = RelationDeclaration("students", [DataTypes.string, DataTypes.integer])
relation_courses = RelationDeclaration("courses", [DataTypes.string])
my_engine.declare_relation_table(relation_students)
my_engine.declare_relation_table(relation_courses)

facts_students_courses = [
    AddFact("students", ["Alice", 20], [DataTypes.string, DataTypes.integer]),
    AddFact("students", ["Bob", 22], [DataTypes.string, DataTypes.integer]),
    AddFact("students", ["Eve", 19], [DataTypes.string, DataTypes.integer]),
    AddFact("courses", ["Math"], [DataTypes.string]),
    AddFact("courses", ["Physics"], [DataTypes.string])
]

for fact in facts_students_courses:
    my_engine.add_fact(fact)

expected_students_output_df = pd.DataFrame([('Alice',20),('Bob',22),('Eve',19)],columns=["col0", "col1"])
expected_courses_output_df = pd.DataFrame(['Math', 'Physics'],columns=["col0"])

curr_students_table = my_engine.table_to_dataframe("students")
pd.testing.assert_frame_equal(curr_students_table, expected_students_output_df)

curr_courses_table = my_engine.table_to_dataframe("courses")
pd.testing.assert_frame_equal(curr_courses_table, expected_courses_output_df)

#| hide
#### remove_fact

In [ ]:
#| export
#| hide
@patch_method
def remove_fact(self: SqliteEngine, 
                fact: RemoveFact # the fact to be removed
                ) -> None:
    """
    Remove a row from an existing sql table, based on `fact`'s terms and types
    """
    num_types = len(fact.type_list)
    col_names = [f"{self._get_col_name(i)}" for i in range(num_types)]
    col_values = [self._convert_relation_term_to_string_or_int(datatype, term) for datatype, term in
                    zip(fact.type_list, fact.term_list)]
    constraint_pairs = zip(col_names, col_values)

    template_dict = {"fact": fact, "constraint_pairs": constraint_pairs}

    sql_template = ("""
    DELETE FROM {{fact.relation_name}} WHERE
    {% for left, right in constraint_pairs %}
        {{left}}={{right}}
        {% if not loop.last %}
            AND
        {% endif %}
    {% endfor %}
    """)

    self._run_sql_from_jinja_template(sql_template, template_dict)

#| hide
#### TEST remove_fact

In [ ]:
#| hide
my_engine = SqliteEngine()

# Declare relation and add facts
relation_remove_fact = RelationDeclaration("yoyo", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation_remove_fact)

facts_remove_fact = [
    AddFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]),
    AddFact("yoyo", [10, "hello"], [DataTypes.integer, DataTypes.string]),
    AddFact("yoyo", [15, "world"], [DataTypes.integer, DataTypes.string])
]

for fact in facts_remove_fact:
    my_engine.add_fact(fact)

expected_output_remove_fact_df = pd.DataFrame([(10, 'hello'), (15, 'world')], columns=["col0", "col1"])

# Remove a fact and compare the result
my_engine.remove_fact(RemoveFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]))
curr_remove_fact_table = my_engine.table_to_dataframe("yoyo")
pd.testing.assert_frame_equal(curr_remove_fact_table, expected_output_remove_fact_df)

# Attempt to remove the same fact again and compare the result
my_engine.remove_fact(RemoveFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]))
curr_remove_fact_table = my_engine.table_to_dataframe("yoyo")
pd.testing.assert_frame_equal(curr_remove_fact_table, expected_output_remove_fact_df)

In [ ]:
#| hide
my_engine = SqliteEngine()

relation = RelationDeclaration("relation", [DataTypes.integer, DataTypes.string, DataTypes.integer])
my_engine.declare_relation_table(relation)

facts = [
    AddFact("relation", [8, "hihi", 3], [DataTypes.integer, DataTypes.string, DataTypes.integer]),
    AddFact("relation", [10, "hello", 2], [DataTypes.integer, DataTypes.string, DataTypes.integer]),
    AddFact("relation", [15, "world", 1], [DataTypes.integer, DataTypes.string, DataTypes.integer])
]

for fact in facts:
    my_engine.add_fact(fact)

# Remove all facts
my_engine.remove_fact(RemoveFact("relation", [8, "hihi", 3], [DataTypes.integer, DataTypes.string, DataTypes.integer]))
my_engine.remove_fact(RemoveFact("relation", [10, "hello", 2], [DataTypes.integer, DataTypes.string, DataTypes.integer]))
my_engine.remove_fact(RemoveFact("relation", [15, "world", 1], [DataTypes.integer, DataTypes.string, DataTypes.integer]))

# Check the empty state
empty_result = my_engine.table_to_dataframe("relation")
expected_empty_output = pd.DataFrame(columns=["col0", "col1", "col2"])
pd.testing.assert_frame_equal(empty_result, expected_empty_output)

# Add back the facts
for fact in facts:
    my_engine.add_fact(fact)

# Check the state after adding back the facts
added_result = my_engine.table_to_dataframe("relation")
expected_output = pd.DataFrame([(8, 'hihi', 3), (10, 'hello', 2), (15, 'world', 1)], columns=["col0", "col1", "col2"])
pd.testing.assert_frame_equal(added_result, expected_output)

#| hide
#### operator_select

In [ ]:
#| export
#| hide
@patch_method
@extract_one_relation
def operator_select(self: SqliteEngine, 
                src_relation: Relation, # the relation from which we select tuples
                constant_variables_info: Set[Tuple[int, Any, DataTypes]], # a set of tuples. each tuple contains the index of the column, the value to select (a constant variable), and the type of the column
                *args: Any
                ) -> Relation: # a filtered relation
    """
    Performs sql WHERE, whose constraints are based on `select_info`
    """
    constant_var_pairs = []
    equal_var_pairs = []

    def _create_new_relation_for_select_result() -> Relation:
        new_term_list = src_relation.term_list
        new_arity = len(new_term_list)
        new_type_list = src_relation.type_list
        new_relation_name = self._create_unique_relation(new_arity, prefix=f"{src_relation.relation_name}{self.SQL_SEPARATOR}{self.SELECT_PREFIX}")
        return Relation(new_relation_name, new_term_list, new_type_list)

    def _extract_constant_variable_pairs() -> List[Tuple[str, str]]:
        """
        generate constraints based on `constant_variables_info`

        @return: column + constant pairs, which are used as constraints for `select`
        """
        for i, value, datatype in constant_variables_info:
            col_name = self._get_col_name(i)
            value = self._convert_relation_term_to_string_or_int(datatype, value)
            constant_var_pairs.append((col_name, value))
        return constant_var_pairs

    def _extract_equal_variable_pairs() -> List[Tuple[str, str]]:
        """
        add constraints based on repeating free variables like a(X,X,8,Y,X) - add constrains to make the 'X's equal.
        notice we only have a single relation here, so we don't have to rename columns.
        for the example above, the `src_relation_var_dict` looks like this:
        {X:[(a,0), (a,1), (a,4)], Y:[(a,3)]}
        in the loop below, we will get, for the variable `X`:
        `first_pair`=`(a,0)`
        `other_pairs`=`[(a,1), (a,4)]`
        and then, our constraints (`equal_var_pairs`) will look like this:
        `[("col0","col1"), ("col0", "col4")]`

        @return: pairs of equal columns
        """
        # get variables in var_dict that repeat - used below to add constraints
        src_relation_var_dict = get_free_var_to_relations_dict({src_relation})
        repeating_vars_in_relation = [(free_var, pairs) for (free_var, pairs) in src_relation_var_dict.items() if (len(pairs) > 1)]

        # convert repeated variables to pairs of columns which should be equal
        for free_var, pairs in repeating_vars_in_relation:
            first_pair, other_pairs = pairs[0], pairs[1:]
            first_index = first_pair[1]
            first_col_name = self._get_col_name(first_index)
            for _, second_index in other_pairs:
                second_col_name = self._get_col_name(second_index)
                equal_var_pairs.append((first_col_name, second_col_name))
        return equal_var_pairs

    selected_relation = _create_new_relation_for_select_result()

    constant_constraints = _extract_constant_variable_pairs()
    equal_var_constraints = _extract_equal_variable_pairs()
    all_constraints = constant_constraints + equal_var_constraints

    template_dict = {"new_rel_name": selected_relation.relation_name, "SELECT": SqliteEngine.SQL_SELECT, "src_rel_name": src_relation.relation_name,
                        "all_constraints": all_constraints}

    sql_template = ("""
    INSERT INTO {{new_rel_name}} {{SELECT}} * FROM {{src_rel_name}}
    {%- if all_constraints %}
    WHERE
        {% for left, right in all_constraints %}
            {{left}}={{right}}
            {% if not loop.last %}
                AND
            {% endif %}
        {% endfor %}
    {%- endif -%}
    """)

    self._run_sql_from_jinja_template(sql_template, template_dict)

    return selected_relation

#| hide
#### TEST operator_select

In [ ]:
#| hide
my_engine = SqliteEngine()

relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

my_engine.declare_relation_table(relation)

facts = [
    AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["banana", "cherry", "date", "elderberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["cherry", "date", "elderberry", "fig"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["date", "elderberry", "fig", "grape"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["elderberry", "fig", "grape", "honeydew"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["fig", "grape", "honeydew", "kiwi"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["grape", "honeydew", "kiwi", "lemon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["honeydew", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["kiwi", "lemon", "mango", "nectarine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["lemon", "mango", "nectarine", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["mango", "nectarine", "pear", "quince"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["nectarine", "orange", "pear", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["orange", "pear", "quince", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "strawberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["quince", "raspberry", "strawberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["raspberry", "strawberry", "pear", "tangerine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["strawberry", "tangerine", "pear", "uva"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["tangerine", "uva", "pear", "violet"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["uva", "violet", "pear", "watermelon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["violet", "watermelon", "pear", "xigua"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "watermelon", "xigua", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["mango", "pear", "quince", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])
]
for fact in facts:
    my_engine.add_fact(fact)

my_engine.operator_select(Relation("wordy", ["pear","Y","Z","W"], [DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]),[(0,'pear',DataTypes.string)])
table_name = '__rgxlog__wordy_select0'

data = [
        ["pear", "kiwi", "lemon", "mango"],
        ["pear", "quince", "raspberry", "strawberry"],
        ["pear", "watermelon", "xigua", "yuzu"],
        ["pear", "quince", "raspberry", "pear"],
        ["pear", "quince", "pear", "pear"]
    ]

expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2", "col3"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name))

table_name = '__rgxlog__wordy_select1'


my_engine.operator_select(Relation("wordy", ["pear","Y","pear","pear"], [DataTypes.string, DataTypes.free_var_name, DataTypes.string, DataTypes.string]),[(0,'pear',DataTypes.string),(2,'pear', DataTypes.string), (3,'pear', DataTypes.string)])
data = [
            ["pear", "quince", "pear", "pear"]
        ]

expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2", "col3"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name))

table_name = '__rgxlog__wordy_select2'
my_engine.operator_select(Relation("wordy", ["X","watermelon","Y","yuzu"], [DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name, DataTypes.string]),[(1,'watermelon',DataTypes.string),(3,'yuzu', DataTypes.string)])
data = [
        ["pear", "watermelon", "xigua", "yuzu"]
    ]

expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2", "col3"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name))

In [ ]:
#| hide
#### operator_join

In [ ]:
#| export
#| hide
@patch_method
def operator_join(self: SqliteEngine, 
            relations: List[Relation], # a list of normal relation
            *args: Any
            ) -> Relation: # a new relation as described above
    """
    note: SQL's inner_join without `IN` is actually cross-join (product), so this covers product as well.
    """
    on_constraints_list: List[Tuple[str, str]] = []
    inner_join_list: List[Tuple[str, str]] = []
    free_var_cols: List[Tuple[str, str]] = []

    def _create_new_relation_for_join_result() -> Relation:
        # get all of the free variables in all of the relations, they'll serve as the terms of the joined relation
        free_var_sets: Sequence[Set] = [get_output_free_var_names(relation) for relation in relations]
        free_vars: Set = set().union(*free_var_sets)
        joined_relation_terms = list(free_vars)

        # get the type list of the joined relation (all of the terms are free variables)
        relation_arity = len(joined_relation_terms)
        relation_types = [DataTypes.free_var_name] * relation_arity

        # declare the joined relation in sql and get its name
        joined_relation_name = self._create_unique_relation(relation_arity, prefix=SqliteEngine.JOIN_PREFIX)

        # create a structured node of the joined relation
        return Relation(joined_relation_name, joined_relation_terms, relation_types)
    @no_type_check
    def _extract_col_names_and_constraints() -> None:
        # iterate over the free_vars and do 2 things:
        for i, free_var in enumerate(joined_relation.term_list):
            free_var_pairs: List[Tuple[Union[Relation, IERelation], int]] = var_dict[free_var]
            first_pair, other_pairs = free_var_pairs[0], free_var_pairs[1:]

            relation_with_free_var, first_index = first_pair
            name_of_relation_with_free_var = relation_temp_names[relation_with_free_var]
            new_col_name = self._get_col_name(i)
            first_col_name = self._get_col_name(first_index)

            # 1. name the new columns. the columns should be named col0, col1, etc.
            old_col_name = f"{name_of_relation_with_free_var}.{first_col_name}"
            new_temp_col_name = f"{new_col_name}"
            free_var_cols.append((old_col_name, new_temp_col_name))

            # 2. create the comparison between all of them, using `ON`
            for (second_relation, second_index) in other_pairs:
                second_col_name = self._get_col_name(second_index)
                name_of_second_relation = relation_temp_names[second_relation]
                first_full_col_name = f"{name_of_relation_with_free_var}.{first_col_name}"
                second_full_col_name = f"{name_of_second_relation}.{second_col_name}"
                on_constraints_list.append((first_full_col_name, second_full_col_name))

    assert len(relations) > 0, "can't join an empty list"
    if len(relations) == 1:
        return relations[0]

    # create a mapping between the relations and their temporary names for sql
    relation_temp_names = {relation: f"table{i}" for (i, relation) in enumerate(relations)}
    var_dict = get_free_var_to_relations_dict(set(relations))

    joined_relation = _create_new_relation_for_join_result()
    _extract_col_names_and_constraints()

    # first relation - used after `FROM`
    first_relation, other_relations = relations[0], relations[1:]

    # every next relation is used after `INNER JOIN`
    for relation in other_relations:
        old_relation_name = relation.relation_name
        new_temp_relation_name = relation_temp_names[relation]
        inner_join_list.append((old_relation_name, new_temp_relation_name))

    template_dict = {"new_rel_name": joined_relation.relation_name, "SELECT": SqliteEngine.SQL_SELECT, "new_columns_names": free_var_cols,
                        "first_rel_name": first_relation.relation_name, "first_rel_temp_name": relation_temp_names[first_relation],
                        "relations_temp_names": inner_join_list, "join_constraints": on_constraints_list}

    sql_template = ("""
    INSERT INTO {{new_rel_name}} {{SELECT}}
    {% for left, right in new_columns_names %}
        {{left}} AS {{right}}
        {% if not loop.last %}
        ,
        {% endif %}
    {% endfor %}

    FROM {{first_rel_name}} AS {{first_rel_temp_name}}
    {% for left, right in relations_temp_names %}
        INNER JOIN {{left}} AS {{right}}
    {% endfor %}

    {%- if join_constraints %}
        ON
        {% for left, right in join_constraints %}
            {{left}}={{right}}
            {% if not loop.last %}
                AND
            {% endif %}
        {% endfor %}
    {%- endif -%}
    """)

    self._run_sql_from_jinja_template(sql_template, template_dict)

    return joined_relation

#| hide
#### operator_project

In [ ]:
#| export
#| hide
@patch_method
@extract_one_relation
def operator_project(self: SqliteEngine, 
                src_relation: Relation, # the relation on which we project
                project_vars: List[str], # a list of variables on which we project
                *args: Any
                ) -> Relation: # the projected relation
    """
    Performs SQL select.
    """
    project_indexes = []
    dest_col_list = []

    def _create_new_relation_for_project_result() -> Relation:
        # get the indexes to project from (in `src_relation`) based on `var_dict`
        var_dict: Dict[str, List[Tuple[Union[Relation, IERelation], int]]] = get_free_var_to_relations_dict({src_relation})

        for var in project_vars:
            var_index_in_src = (var_dict[var][0][1])
            project_indexes.append(var_index_in_src)

        src_type_list = src_relation.type_list
        new_type_list = [src_type_list[i] for i in project_indexes]
        new_arity = len(project_vars)
        new_relation_name = self._create_unique_relation(new_arity,
                                                            prefix=f"{src_relation.relation_name}{SqliteEngine.SQL_SEPARATOR}{SqliteEngine.PROJECT_PREFIX}")
        return Relation(new_relation_name, project_vars, new_type_list)

    def _extract_project_col_names() -> None:
        for new_col_num, src_col_num in enumerate(project_indexes):
            new_col = self._get_col_name(new_col_num)
            src_col = self._get_col_name(src_col_num)
            if new_col == src_col:
                # this prevents selecting "colX AS colX", for aesthetic reasons
                dest_col_list.append(src_col)
            else:
                dest_col_list.append(f"{src_col} AS {new_col}")

    new_relation = _create_new_relation_for_project_result()
    _extract_project_col_names()

    sql_command = (f"INSERT INTO {new_relation.relation_name} {SqliteEngine.SQL_SELECT} {', '.join(dest_col_list)}"
                    f" FROM {src_relation.relation_name}")

    self._run_sql(sql_command)
    return new_relation

#| hide
#### TEST operator_project

In [ ]:
#| hide
my_engine = SqliteEngine()

relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

my_engine.declare_relation_table(relation)

facts = [
    AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["banana", "cherry", "date", "elderberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["cherry", "date", "elderberry", "fig"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["date", "elderberry", "fig", "grape"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["elderberry", "fig", "grape", "honeydew"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["fig", "grape", "honeydew", "kiwi"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["grape", "honeydew", "kiwi", "lemon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["honeydew", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["kiwi", "lemon", "mango", "nectarine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["lemon", "mango", "nectarine", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["mango", "nectarine", "pear", "quince"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["nectarine", "orange", "pear", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["orange", "pear", "quince", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "strawberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["quince", "raspberry", "strawberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["raspberry", "strawberry", "pear", "tangerine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["strawberry", "tangerine", "pear", "uva"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["tangerine", "uva", "pear", "violet"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["uva", "violet", "pear", "watermelon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["violet", "watermelon", "pear", "xigua"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "watermelon", "xigua", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["mango", "pear", "quince", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["X", "quince", "pear", "Y"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["violet", "quince", "pear", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["X", "quince", "pear", "W"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),

]
for fact in facts:
    my_engine.add_fact(fact)


project_relation = my_engine.operator_select(Relation("wordy", ["pear","Y","Z","W"], [DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]),[(0,'pear',DataTypes.string)])
table_name = my_engine.operator_project(project_relation, ['Y','Z','W'])

data = [
        ["kiwi", "lemon", "mango"],
        ["quince", "raspberry", "strawberry"],
        ["watermelon", "xigua", "yuzu"],
        ["quince", "raspberry", "pear"],
        ["quince", "pear", "pear"]
    ]
expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))

project_relation = my_engine.operator_select(Relation("wordy", ["pear","quince","Z","W"],
            [DataTypes.string, DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name]),
            [(0,'pear',DataTypes.string),(1,'quince',DataTypes.string)])

table_name = my_engine.operator_project(project_relation, ['Z','W'])

data = [
        ["raspberry", "strawberry"],
        ["raspberry", "pear"],
        ["pear", "pear"]
    ]

expected_df = pd.DataFrame(data, columns=["col0", "col1"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))

project_relation = my_engine.operator_select(Relation("wordy", ["X","quince","pear","W"],
            [DataTypes.free_var_name, DataTypes.string, DataTypes.string, DataTypes.free_var_name]),
            [(2,'pear',DataTypes.string),(1,'quince',DataTypes.string)])

table_name = my_engine.operator_project(project_relation, ['X','W'])

data = [
        ["pear", "pear"],
        ["X", "Y"],
        ["violet", "yuzu"],
        ["X", "W"]
]

expected_df = pd.DataFrame(data, columns=["col0", "col1"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))

#| hide
#### operator_union

In [ ]:
#| export
#| hide
@patch_method
def operator_union(self: SqliteEngine, 
                relations: List[Relation], # a list of relations to unite
                *args: Any
                ) -> Relation: # the united relation
    union_list: List[str] = []

    def _create_new_relation_for_union() -> Relation:
        new_relation_name = self._create_unique_relation(len(relations[0].term_list), prefix=SqliteEngine.UNION_PREFIX)
        new_term_list = relations[0].term_list
        new_type_list = relations[0].type_list
        return Relation(new_relation_name, new_term_list, new_type_list)

    def _extract_union_selections() -> None:
        """
        create the union command by iterating over the relations and finding the index of each term
        @return: None
        """

        for relation in relations:
            # we assume the same order in the source and the destination, so no need to use 'AS'
            selection_list = [self._get_col_name(col_index) for col_index in range(len(united_relation.term_list))]

            # render a jinja template into an SQL select
            relation_string_template = '{{SELECT}} {{ selected_cols | join(", ") }} FROM {{rel_name}}'
            template_dict = {"SELECT": SqliteEngine.SQL_SELECT, "selected_cols": selection_list, "rel_name": relation.relation_name}
            rendered_relation_string = Template(strip_lines(relation_string_template)).render(**template_dict)
            union_list.append(rendered_relation_string)

    new_arity = len(relations)
    assert new_arity > 0, "cannot perform union on an empty list"
    if new_arity == 1:
        return relations[0]

    united_relation = _create_new_relation_for_union()
    _extract_union_selections()

    sql_command = f"INSERT INTO {united_relation.relation_name} {' UNION '.join(union_list)}"

    self._run_sql(sql_command)
    return united_relation

#| hide
#### TEST operator_union

In [ ]:
#| hide
my_engine = SqliteEngine()

relation1 = RelationDeclaration("relation1", [DataTypes.integer, DataTypes.integer])
my_engine.declare_relation_table(relation1)

facts1 = [
    AddFact("relation1", [1, 2], [DataTypes.integer, DataTypes.integer]),
    AddFact("relation1", [3, 4], [DataTypes.integer, DataTypes.integer]),
    AddFact("relation1", [5, 6], [DataTypes.integer, DataTypes.integer]),
    AddFact("relation1", [7, 8], [DataTypes.integer, DataTypes.integer]),
    AddFact("relation1", [9, 10], [DataTypes.integer, DataTypes.integer])
]
for fact in facts1:
    my_engine.add_fact(fact)

relation2 = RelationDeclaration("relation2", [DataTypes.string, DataTypes.string])
my_engine.declare_relation_table(relation2)

facts2 = [
    AddFact("relation2", ["apple", "banana"], [DataTypes.string, DataTypes.string]),
    AddFact("relation2", ["cherry", "date"], [DataTypes.string, DataTypes.string]),
    AddFact("relation2", ["elderberry", "fig"], [DataTypes.string, DataTypes.string]),
    AddFact("relation2", ["grape", "honeydew"], [DataTypes.string, DataTypes.string]),
    AddFact("relation2", ["kiwi", "lemon"], [DataTypes.string, DataTypes.string])
]
for fact in facts2:
    my_engine.add_fact(fact)

relation3 = RelationDeclaration("relation3", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation3)

facts3 = [
    AddFact("relation3", [1, "apple"], [DataTypes.integer, DataTypes.string]),
    AddFact("relation3", [2, "banana"], [DataTypes.integer, DataTypes.string]),
    AddFact("relation3", [3, "cherry"], [DataTypes.integer, DataTypes.string]),
    AddFact("relation3", [4, "date"], [DataTypes.integer, DataTypes.string]),
    AddFact("relation3", [5, "elderberry"], [DataTypes.integer, DataTypes.string])
]
for fact in facts3:
    my_engine.add_fact(fact)

rel1 = Relation("relation1", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name])
rel2 = Relation("relation2", ["Z", "W"], [DataTypes.free_var_name, DataTypes.free_var_name])
rel3 = Relation("relation3", ["Z", "W"], [DataTypes.free_var_name, DataTypes.free_var_name])


table_name = my_engine.operator_union([rel1,rel2])
data = [
            [1, 2],
            [3, 4],
            [5, 6],
            [7, 8],
            [9, 10],
            ["apple", "banana"],
            ["cherry", "date"],
            ["elderberry", "fig"],
            ["grape", "honeydew"],
            ["kiwi", "lemon"]
        ]
expected_df = pd.DataFrame(data, columns=["col0", "col1"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))    

table_name = my_engine.operator_union([rel1,rel2, rel3])
data = [
            [1, 2],
            [1, "apple"],
            [2, "banana"],
            [3, 4],
            [3, "cherry"],
            [4, "date"],
            [5, 6],
            [5, "elderberry"],
            [7, 8],
            [9, 10],
            ["apple", "banana"],
            ["cherry", "date"],
            ["elderberry", "fig"],
            ["grape", "honeydew"],
            ["kiwi", "lemon"]
        ]
expected_df = pd.DataFrame(data, columns=["col0", "col1"])
assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))    

#| hide
#### operator_copy

In [ ]:
#| export
#| hide
@patch_method
@extract_one_relation
def operator_copy(self: SqliteEngine, src_rel: Relation, output_relation: Optional[Relation] = None, *args: Any) -> Relation:
    """
    See `RgxlogEngineBase.operator_copy` for explanation
    """
    src_rel_name = src_rel.relation_name
    if output_relation:
        dest_rel_name = output_relation.relation_name

        # check if the relation already exists
        if self.is_table_exists(dest_rel_name):
            self.clear_relation(dest_rel_name)
        else:
            dest_decl_rel = RelationDeclaration(dest_rel_name, src_rel.type_list)
            self.declare_relation_table(dest_decl_rel)

    else:
        dest_rel_name = self._create_unique_relation(arity=len(src_rel.type_list),
                                                        prefix=f"{src_rel_name}{SqliteEngine.SQL_SEPARATOR}{SqliteEngine.COPY_PREFIX}")

    dest_rel = Relation(dest_rel_name, src_rel.term_list, src_rel.type_list)

    # sql part
    sql_command = f"INSERT INTO {dest_rel_name} {SqliteEngine.SQL_SELECT} * FROM {src_rel_name}"
    self._run_sql(sql_command)

    return dest_rel


In [ ]:
show_doc(SqliteEngine.operator_copy)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L945){target="_blank" style="float:right; font-size:smaller"}

### operator_copy

>      operator_copy (src_rel:rgxlog.ast_node_types.Relation,
>                     output_relation:Union[rgxlog.ast_node_types.Relation,NoneT
>                     ype]=None, *args:Any)

See `RgxlogEngineBase.operator_copy` for explanation

#| hide
#### query

In [ ]:
#| export
#| hide
@patch_method
def query(self: SqliteEngine, 
                query: Query, # the query to be performed
                allow_duplicates: bool = False # if True, query result may contain duplicate values
                ) -> List[Tuple]: # a query results which is True, False, or a list of tuples
    """
    Outputs a preformatted query result, e.g. [("a",5),("b",6)].
    notice that `query` isn't a string; it's a `Query` object which inherits from `Relation`.
    for example, parsing the string `?excellent("bill","ted")` yields the following `Query`:

    ```prolog
    relation_name = excellent
    term_list = ["bill", "ted"]
    type_list = [DataType.string, DataType.string]
    ```
    """
    query_free_var_indexes = self._get_free_variable_indexes(query.type_list)
    has_free_vars = bool(query_free_var_indexes)
    select_info = query.get_select_cols_values_and_types()

    # create temporary tables for the select/project, and delete them
    selected_relation = self.operator_select(query, select_info)
    selected_relation_name = selected_relation.relation_name

    free_var_names_for_project = [term for term, term_type in zip(query.term_list, query.type_list)
                                    if term_type is DataTypes.free_var_name]
    if has_free_vars:
        projected_relation_name = self.operator_project(selected_relation, free_var_names_for_project).relation_name
    else:
        projected_relation_name = selected_relation_name

    query_result = self._run_sql(f"{SqliteEngine.SQL_SELECT} * FROM {projected_relation_name}", do_commit=True)

    self.remove_table(selected_relation_name)
    self.remove_table(projected_relation_name)

    # we need to convert values of type `True` and `Span` into their true form. `False` is already in its true form.
    if (not has_free_vars) and query_result != FALSE_VALUE:
        query_result = TRUE_VALUE

    spanned_query_result = self._convert_strings_to_spans_in_query_result(query_result)

    return spanned_query_result

#| hide
#### TEST query

In [ ]:
#| hide
my_engine = SqliteEngine()

relation = RelationDeclaration("people", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation)

facts = [
    AddFact("people", [1, "Alice"], [DataTypes.integer, DataTypes.string]),
    AddFact("people", [2, "Bob"], [DataTypes.integer, DataTypes.string]),
    AddFact("people", [3, "Charlie"], [DataTypes.integer, DataTypes.string])
]

for fact in facts:
    my_engine.add_fact(fact)

query_result = my_engine.query(Query("people", ["ID", "Name"],
                                        [DataTypes.free_var_name, DataTypes.free_var_name]))

expected_output = [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie")
]

assert expected_output == query_result

courses_relation = RelationDeclaration("courses", [DataTypes.string, DataTypes.integer])
my_engine.declare_relation_table(courses_relation)

facts = [
    AddFact("courses", ["Math", 101], [DataTypes.string, DataTypes.integer]),
    AddFact("courses", ["Science", 202], [DataTypes.string, DataTypes.integer]),
    AddFact("courses", ["History", 303], [DataTypes.string, DataTypes.integer])
]

for fact in facts:
    my_engine.add_fact(fact)

query_result = my_engine.query(Query("courses", ["Name", "Number"],
                                        [DataTypes.free_var_name, DataTypes.free_var_name]))

expected_output = [
    ("Math", 101),
    ("Science", 202),
    ("History", 303)
]

assert expected_output == query_result

In [ ]:
#| hide
my_engine = SqliteEngine()

relation1 = RelationDeclaration("important_years", [DataTypes.integer])
relation2 = RelationDeclaration("courses", [DataTypes.string, DataTypes.integer])

my_engine.declare_relation_table(relation1)
my_engine.declare_relation_table(relation2)


facts = [
    AddFact("important_years",[2000],[DataTypes.integer]),
    AddFact("important_years",[1900],[DataTypes.integer]),
    AddFact("important_years",[1800],[DataTypes.integer]),
    AddFact("important_years",[1700],[DataTypes.integer]),
    AddFact("important_years",[1600],[DataTypes.integer]),
    AddFact("courses", ["Math", 2000], [DataTypes.string, DataTypes.integer]),
    AddFact("courses", ["Science", 1900], [DataTypes.string, DataTypes.integer]),
    AddFact("courses", ["History", 1800], [DataTypes.string, DataTypes.integer])
]

for fact in facts:
    my_engine.add_fact(fact)

query_result = my_engine.query(Query("courses", ["Name", 1900],
                                        [DataTypes.free_var_name, DataTypes.integer]))

assert query_result == [('Science',)]

In [ ]:
#| hide
my_engine = SqliteEngine()

relation1 = RelationDeclaration("people", [DataTypes.integer, DataTypes.string])
relation2 = RelationDeclaration("courses", [DataTypes.string, DataTypes.string, DataTypes.integer])
relation3 = RelationDeclaration("enrollments", [DataTypes.integer, DataTypes.string])

my_engine.declare_relation_table(relation1)
my_engine.declare_relation_table(relation2)
my_engine.declare_relation_table(relation3)

facts = [
    AddFact("people", [1, "Alice"], [DataTypes.integer, DataTypes.string]),
    AddFact("people", [2, "Bob"], [DataTypes.integer, DataTypes.string]),
    AddFact("people", [3, "Charlie"], [DataTypes.integer, DataTypes.string]),
    AddFact("courses", ["Math", "ProfA", 101], [DataTypes.string, DataTypes.string, DataTypes.integer]),
    AddFact("courses", ["Science", "ProfB", 202], [DataTypes.string, DataTypes.string, DataTypes.integer]),
    AddFact("courses", ["History", "ProfC", 303], [DataTypes.string, DataTypes.string, DataTypes.integer]),
    AddFact("enrollments", [1, "Math"], [DataTypes.integer, DataTypes.string]),
    AddFact("enrollments", [2, "Science"], [DataTypes.integer, DataTypes.string]),
    AddFact("enrollments", [3, "History"], [DataTypes.integer, DataTypes.string]),
]

for fact in facts:
    my_engine.add_fact(fact)

query_result = my_engine.query(Query("enrollments", ["X", "Y"],
                                        [DataTypes.free_var_name, DataTypes.free_var_name]))

expected_output = [
    (1, "Math"),
    (2, "Science"),
    (3, "History")
]

assert query_result == expected_output

In [ ]:
#| hide
my_engine = SqliteEngine()
relation = RelationDeclaration("common_relation", [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string])

my_engine.declare_relation_table(relation)

facts = [
    AddFact("common_relation", [1, 100, "Apple", 50, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [2, 200, "Banana", 40, "Yellow"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [3, 300, "Cherry", 30, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [4, 400, "Date", 20, "Brown"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [5, 500, "Elderberry", 10, "Purple"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [6, 200, "Fig", 40, "Green"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [7, 400, "Grape", 20, "Purple"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [8, 100, "Apple", 50, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [9, 500, "Elderberry", 10, "Purple"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
    AddFact("common_relation", [10, 300, "Cherry", 30, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string])
]

for fact in facts:
    my_engine.add_fact(fact)

query_result = my_engine.query(Query("common_relation", ["X", 200, "Y", "Z", "W"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))


expected_result = [
    (2, "Banana", 40, "Yellow"),
    (6, "Fig", 40, "Green")
]

assert query_result == expected_result

query_result = my_engine.query(Query("common_relation", ["X", 200, "Y", 40, "W"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name]))

expected_result = [
    (2, "Banana", "Yellow"),
    (6, "Fig", "Green")
]

assert query_result == expected_result


query_result = my_engine.query(Query("common_relation", ["X", "Y", "Y", 40, "W"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name]))

expected_result = []

assert query_result == expected_result


query_result = my_engine.query(Query("common_relation", ["X", 100, "Apple", 50, "W"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.free_var_name]))

expected_result = [
    (1, "Red"),
    (8, "Red")
]
assert query_result == expected_result

query_result = my_engine.query(Query("common_relation", ["X", 100, "Apple", 50, "Red"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]))

expected_result = [
    (1,),
    (8,)
]
assert query_result == expected_result

In [ ]:
#| hide
my_engine = SqliteEngine()

relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

my_engine.declare_relation_table(relation)

facts = [
    AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["banana", "cherry", "date", "elderberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["cherry", "date", "elderberry", "fig"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["date", "elderberry", "fig", "grape"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["elderberry", "fig", "grape", "honeydew"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["fig", "grape", "honeydew", "kiwi"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["grape", "honeydew", "kiwi", "lemon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["honeydew", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["kiwi", "lemon", "mango", "nectarine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["lemon", "mango", "nectarine", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["mango", "nectarine", "pear", "quince"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["nectarine", "orange", "pear", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["orange", "pear", "quince", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "strawberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["quince", "raspberry", "strawberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["raspberry", "strawberry", "pear", "tangerine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["strawberry", "tangerine", "pear", "uva"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["tangerine", "uva", "pear", "violet"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["uva", "violet", "pear", "watermelon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["violet", "watermelon", "pear", "xigua"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "watermelon", "xigua", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["mango", "pear", "quince", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])
]

for fact in facts:
    my_engine.add_fact(fact)

query_result = my_engine.query(Query("wordy", ["X", "kiwi", "Y", "Z"], [DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name]))

expected_result = [
    ("honeydew",  "lemon", "mango"),
    ("pear", "lemon", "mango")
]

assert query_result == expected_result

query_result = my_engine.query(Query("wordy", ["X", "Y", "X", "Z"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))

expected_result = [
    ("pear", "quince", "pear", "pear"),
]
assert query_result == expected_result

query_result = my_engine.query(Query("wordy", ["X", "Y", "Z", "X"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))

expected_result = [
("pear", "quince", "raspberry", "pear"),
("pear", "quince", "pear", "pear")

]
assert query_result == expected_result

query_result = my_engine.query(Query("wordy", ["X", "Y", "raspberry", "X"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name]))


expected_result = [
    ("pear", "quince", "pear")
]

assert query_result == expected_result


query_result = my_engine.query(Query("wordy", ["mango", "X", "quince", "X"], [DataTypes.string, DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name]))


expected_result = [
    ("pear", "pear")
]

assert query_result == expected_result

query_result = my_engine.query(Query("wordy", ["pear", "quince", "X", "Y"], [DataTypes.string, DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name]))


expected_result = [
    ("raspberry", "strawberry"),
    ("raspberry", "pear"),
    ("pear", "pear")
]

assert query_result == expected_result

#| hide
#### compute_ie_relation

In [ ]:
#| export
#| hide
@patch_method
def _get_all_relation_tuples(self: SqliteEngine, 
                             relation: Relation # a relation to be queried
                             ) -> List[Tuple]: # all the tuples of 'relation' as a list of tuples

    relation_name = relation.relation_name
    relation_arity = len(relation.term_list)

    # in order to get all of the tuples of the relation, we'll query the relation with a query that only has
    # free variable terms, where each one of the free variables has a different name
    # for example for the relation 'Parent(str, str)', we'll construct the query '?Parent(X0, X1)'
    query_relation_name = relation_name
    query_terms = [f'X{i}' for i in range(relation_arity)]
    query_term_types = [DataTypes.free_var_name] * relation_arity
    query = Query(query_relation_name, query_terms, query_term_types)

    # query the relation using the query we've constructed, and return the result
    all_relation_tuples = self.query(query)
    return all_relation_tuples

In [ ]:
#| export
#| hide
@patch_method
def compute_ie_relation(self: SqliteEngine, 
                ie_relation: IERelation, # an ie relation that determines the input and output terms of the ie function
                ie_func: IEFunction, # the ie function that will be used to compute the ie relation
                bounding_relation: Optional[Relation] # a relation that contains the inputs for ie_funcs. the actual input needs to be queried from it
                ) -> Relation: # a normal relation that contains all of the resulting tuples in the rgxlog engine
    """
    Computes an information extraction relation, returning the result as a normal relation.
    for more details see RgxlogEngineBase.compute_ie_relation.
    notice comments below regarding constants
    """

    def _looks_like_span(checked_value: Any) -> bool:
        """
        checks whether `term` is a tuple of 2 numbers
        @param checked_value: the value to check
        @return: True/False according to the above description
        """
        if isinstance(checked_value, tuple) and len(checked_value) == 2:
            try:
                _, _ = int(checked_value[0]), int(checked_value[1])
                return True
            except ValueError:
                # an item in the tuple could not be converted to int
                return False
        return False

    def _get_all_ie_function_inputs() -> List[tuple]:
        # define the ie input relation
        if bounding_relation is None:
            # special case where the ie relation is the first rule body relation
            # in this case, the ie input relation is defined exclusively by constant terms, i.e, by a single tuple
            # add that tuple as a fact to the input relation
            # create the input relation for the ie function, and also declare it inside SQL
            all_ie_inputs = [tuple(ie_relation.input_term_list)]
        else:
            # get a list of inputs to the ie function - some of them may be constants
            inputs_without_constants = self._get_all_relation_tuples(bounding_relation)
            all_ie_inputs = []
            for bounded_input in inputs_without_constants:
                result_input_list = []
                for term, datatype in zip(ie_relation.input_term_list, ie_relation.input_type_list):
                    if datatype is DataTypes.free_var_name:
                        # add value from `bounded_input`
                        result_input_list.append(bounded_input[bounding_relation.get_index_of_free_var(term)])
                    else:
                        # add a constant from the ie_relation's input
                        result_input_list.append(term)
                all_ie_inputs.append(tuple(result_input_list))
        return all_ie_inputs

    def _format_ie_output(raw_ie_output):
        # the output should be a tuple, but if a single value is returned, we accept it as well
        if isinstance(raw_ie_output, (str, int, Span)):
            return [raw_ie_output]
        else:
            # the user is allowed to represent a span in an ie output as a tuple of length 2
            # convert said tuples to spans
            return [Span(int(term[0]), int(term[1])) if _looks_like_span(term) else term for term in list(raw_ie_output)]

    def _run_ie_function_and_add_outputs_as_facts():
        # run the ie function on each input and process the outputs
        for ie_input in ie_inputs:
            # run the ie function on the input, resulting in a list of tuples
            ie_outputs = ie_func.ie_function(*ie_input)
            # process each ie output and add it to the output relation
            for ie_output in ie_outputs:
                spanned_ie_output = _format_ie_output(ie_output)

                # assert the ie output is properly typed
                self._assert_ie_output_properly_typed(ie_input, list(ie_input) + spanned_ie_output, ie_output_schema, ie_relation)

                # add the output as a fact to the output relation
                # notice - repetitions are ignored here (results are in a set)
                if len(spanned_ie_output) != 0:
                    output_fact = AddFact(output_relation.relation_name, list(ie_input) + spanned_ie_output, list(ie_output_schema))
                    self.add_fact(output_fact)

    ie_relation_name = ie_relation.relation_name
    # create the output relation for the ie function, and also declare it inside SQL
    output_relation_arity = len(ie_relation.input_term_list) + len(ie_relation.output_term_list)
    output_relation_name = self._create_unique_relation(output_relation_arity,
                                                        prefix=f'{ie_relation_name}{SqliteEngine.SQL_SEPARATOR}output')
    output_relation = Relation(output_relation_name, ie_relation.get_term_list(), ie_relation.get_type_list())

    ie_inputs = _get_all_ie_function_inputs()

    ie_output_schema = ie_func.get_output_types(output_relation_arity)

    _run_ie_function_and_add_outputs_as_facts()

    return output_relation

In [ ]:
show_doc(SqliteEngine.compute_ie_relation)

---

### SqliteEngine.compute_ie_relation

>      SqliteEngine.compute_ie_relation
>                                        (ie_relation:rgxlog.ast_node_types.IERe
>                                        lation,
>                                        ie_func:rgxlog.ie_function.IEFunction, 
>                                        bounding_relation:Union[rgxlog.ast_node
>                                        _types.Relation,NoneType])

Computes an information extraction relation, returning the result as a normal relation.
for more details see RgxlogEngineBase.compute_ie_relation.
notice comments below regarding constants

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| ie_relation | IERelation | an ie relation that determines the input and output terms of the ie function |
| ie_func | IEFunction | the ie function that will be used to compute the ie relation |
| bounding_relation | Optional[Relation] | a relation that contains the inputs for ie_funcs. the actual input needs to be queried from it |
| **Returns** | **Relation** | **a normal relation that contains all of the resulting tuples in the rgxlog engine** |

In [ ]:
show_doc(SqliteEngine.add_fact)

---

### SqliteEngine.add_fact

>      SqliteEngine.add_fact (fact:rgxlog.ast_node_types.AddFact)

Add a row into an existing sql table, based on `fact`'s terms and types

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| fact | AddFact | the fact to be added |
| **Returns** | **None** |  |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
my_engine = SqliteEngine()

relation = RelationDeclaration("yoyo", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation)

facts = [
    AddFact("yoyo", [10, "hello"], [DataTypes.integer, DataTypes.string]),
    AddFact("yoyo", [15, "world"], [DataTypes.integer, DataTypes.string])
]

for fact in facts:
    my_engine.add_fact(fact)

my_engine.table_to_dataframe("yoyo")

,col0,col1
0,10,hello
1,15,world


:::

In [ ]:
show_doc(SqliteEngine.remove_fact)

---

### SqliteEngine.remove_fact

>      SqliteEngine.remove_fact (fact:rgxlog.ast_node_types.RemoveFact)

Remove a row from an existing sql table, based on `fact`'s terms and types

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| fact | RemoveFact | the fact to be removed |
| **Returns** | **None** |  |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
my_engine = SqliteEngine()

relation = RelationDeclaration("yoyo", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation)

facts = [
    AddFact("yoyo", [10, "hello"], [DataTypes.integer, DataTypes.string]),
    AddFact("yoyo", [15, "world"], [DataTypes.integer, DataTypes.string])
]

for fact in facts:
    my_engine.add_fact(fact)
my_engine.table_to_dataframe("yoyo")

,col0,col1
0,10,hello
1,15,world


In [ ]:
my_engine.remove_fact(RemoveFact("yoyo", [15, "world"], [DataTypes.integer, DataTypes.string]))
my_engine.table_to_dataframe("yoyo")

,col0,col1
0,10,hello


:::

In [ ]:
show_doc(SqliteEngine.query)

---

### SqliteEngine.query

>      SqliteEngine.query (query:rgxlog.ast_node_types.Query,
>                          allow_duplicates:bool=False)

Outputs a preformatted query result, e.g. [("a",5),("b",6)].
notice that `query` isn't a string; it's a `Query` object which inherits from `Relation`.
for example, parsing the string `?excellent("bill","ted")` yields the following `Query`:

```prolog
relation_name = excellent
term_list = ["bill", "ted"]
type_list = [DataType.string, DataType.string]
```

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| query | Query |  | the query to be performed |
| allow_duplicates | bool | False | if True, query result may contain duplicate values |
| **Returns** | **List[Tuple]** |  | **a query results which is True, False, or a list of tuples** |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
relation = RelationDeclaration("people", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation)

facts = [
    AddFact("people", [1, "Alice"], [DataTypes.integer, DataTypes.string]),
    AddFact("people", [2, "Bob"], [DataTypes.integer, DataTypes.string])
]

for fact in facts:
    my_engine.add_fact(fact)

query_result = my_engine.query(Query("people", ["ID", "Name"],
                                        [DataTypes.free_var_name, DataTypes.free_var_name]))

print(query_result)

[(1, 'Alice'), (2, 'Bob')]


:::

In [ ]:
show_doc(SqliteEngine.declare_relation_table)

---

### SqliteEngine.declare_relation_table

>      SqliteEngine.declare_relation_table
>                                           (relation_decl:rgxlog.ast_node_types
>                                           .RelationDeclaration)

Declares a relation as an SQL table, whose types are named t0, t1, ...
if the relation is already declared, do nothing.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relation_decl | RelationDeclaration | the declaration info |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
my_engine = SqliteEngine()

relation = RelationDeclaration("people", [DataTypes.integer, DataTypes.string])
my_engine.declare_relation_table(relation)

my_engine.is_table_exists("people")

True

:::

In [ ]:
show_doc(SqliteEngine.operator_select)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L661){target="_blank" style="float:right; font-size:smaller"}

### operator_select

>      operator_select (src_relation:rgxlog.ast_node_types.Relation, constant_va
>                       riables_info:Set[Tuple[int,Any,rgxlog.primitive_types.Da
>                       taTypes]], *args:Any)

Performs sql WHERE, whose constraints are based on `select_info`

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| src_relation | Relation | the relation from which we select tuples |
| constant_variables_info | Set[Tuple[int, Any, DataTypes]] | a set of tuples. each tuple contains the index of the column, the value to select (a constant variable), and the type of the column |
| args | Any |  |
| **Returns** | **Relation** | **a filtered relation** |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
my_engine = SqliteEngine()

relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

my_engine.declare_relation_table(relation)
facts = [
    AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
]

for fact in facts:
    my_engine.add_fact(fact)

relation_res = my_engine.operator_select(Relation("wordy", ["pear","Y","Z","W"], [DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]),[(0,'pear',DataTypes.string)])
print(my_engine.table_to_dataframe(relation_res.relation_name))

   col0    col1       col2   col3
0  pear    kiwi      lemon  mango
1  pear  quince  raspberry   pear


:::

In [ ]:
show_doc(SqliteEngine.operator_join)

---

### SqliteEngine.operator_join

>      SqliteEngine.operator_join
>                                  (relations:List[rgxlog.ast_node_types.Relatio
>                                  n], *args:Any)

note: SQL's inner_join without `IN` is actually cross-join (product), so this covers product as well.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relations | List[Relation] | a list of normal relation |
| args | Any |  |
| **Returns** | **Relation** | **a new relation as described above** |

In [ ]:
show_doc(SqliteEngine.operator_project)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/rgxlog/engine.py#L854){target="_blank" style="float:right; font-size:smaller"}

### operator_project

>      operator_project (src_relation:rgxlog.ast_node_types.Relation,
>                        project_vars:List[str], *args:Any)

Performs SQL select.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| src_relation | Relation | the relation on which we project |
| project_vars | List[str] | a list of variables on which we project |
| args | Any |  |
| **Returns** | **Relation** | **the projected relation** |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
my_engine = SqliteEngine()

relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

my_engine.declare_relation_table(relation)
facts = [
    AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
    AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
]

for fact in facts:
    my_engine.add_fact(fact)

project_relation = my_engine.operator_select(Relation("wordy", ["pear","Y","Z","W"], [DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]),[(0,'pear',DataTypes.string)])
table_name = my_engine.operator_project(project_relation, ['Y','Z','W']).relation_name
my_engine.table_to_dataframe(table_name)

,col0,col1,col2
0,kiwi,lemon,mango
1,quince,raspberry,pear


:::

In [ ]:
show_doc(SqliteEngine.operator_union)

---

### SqliteEngine.operator_union

>      SqliteEngine.operator_union
>                                   (relations:List[rgxlog.ast_node_types.Relati
>                                   on], *args:Any)

a rule can be defined multiple times as long as it shares the same rule head. To illustrate this, let's consider the following example:

Suppose we have the rules:
```prolog
A(X,Y) <- B(X,Y)
A(X,Y) <- C(X,Y)
```
If we execute the query `?A(X,Y)`, the engine will produce results that include tuples from both `B(X,Y)` and `C(X,Y)` relations. 

Here's how the engine handles this scenario: <br>
1. **Rule Matching**: The engine identifies all the rules that have the same rule head (`A(X,Y)`) as the queried relation. <br>
2. **Rule Execution**: For each matched rule, the engine executes all the steps described earlier, such as assigning temporary names, performing joins, and creating intermediate relations, selecting and filtering. <br>
3. **Intermediate Relations**: At this stage, we have intermediate relations generated for each rule, containing the relevant tuples based on the rule's body. <br>
4. **Operator Union**: To consolidate the results from all the rules into a single unified relation, the operator union is applied. This operation combines the tuples from all the intermediate relations. <br>

The `operator_union` ensures that the final unified relation contains all the tuples from the individual intermediate relations, effectively merging the results obtained from different rule instances with the same rule head.
In summary, the engine processes each rule with the same rule head as the queried relation, performs the necessary steps for each rule individually, and then applies the operator union to merge the intermediate relations into a single unified relation, ensuring all relevant tuples are included in the final result.        @note: we assume that all the relations have same free_vars in the same order.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| relations | List[Relation] | a list of relations to unite |
| args | Any |  |
| **Returns** | **Relation** | **the united relation** |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
my_engine = SqliteEngine()

relation1 = RelationDeclaration("relation1", [DataTypes.integer, DataTypes.integer])
my_engine.declare_relation_table(relation1)

facts1 = [
    AddFact("relation1", ["two", 2], [DataTypes.string, DataTypes.integer]),
    AddFact("relation1", ["four", 4], [DataTypes.string, DataTypes.integer]),
    AddFact("relation1", ["six", 6], [DataTypes.string, DataTypes.integer]),
]
for fact in facts1:
    my_engine.add_fact(fact)

relation2 = RelationDeclaration("relation2", [DataTypes.string, DataTypes.string])
my_engine.declare_relation_table(relation2)
facts2 = [
    AddFact("relation2", ["one", 1], [DataTypes.string, DataTypes.integer]),
]
for fact in facts2:
    my_engine.add_fact(fact)
rel1 = Relation("relation1", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name])
rel2 = Relation("relation2", ["Z", "W"], [DataTypes.free_var_name, DataTypes.free_var_name])
table_name = my_engine.operator_union([rel1,rel2]).relation_name
my_engine.table_to_dataframe(table_name)

,col0,col1
0,four,4
1,one,1
2,six,6
3,two,2


:::

In [ ]:
#| export
if __name__ == "__main__":
    my_engine = SqliteEngine()
    print("hello world")

    # add relation
    my_relation = RelationDeclaration("yoyo", [DataTypes.integer, DataTypes.string])
    my_engine.declare_relation_table(my_relation)

    # add fact
    my_fact = AddFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string])
    my_engine.add_fact(my_fact)
    my_engine.add_fact(my_fact)

    my_query = Query("yoyo", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name])
    print(my_engine.query(my_query))

    print(my_engine.get_table_len("yoyo"))
    my_engine.print_sql()


hello world
[(8, 'hihi')]
2
Table: yoyo
(8, 'hihi')
(8, 'hihi')

